In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
import pandas as pd
import json
from datetime import datetime
from sklearn import feature_selection, model_selection, preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

import mytrain_lib as ml

import importlib

torch.manual_seed(0)
import random
random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
path_train      = 'F://TFG//datasets//data_train//'
path_graphs     = 'F://TFG//graphs//plot_results//'
path_results    = 'F://TFG//results//'
path_logs     = path_results+'logs//'

In [ ]:
dataDF = pd.read_csv(path_train+'training_features_DF.csv',sep=';',index_col='wyId')
dataDF.head(3)

#### Links:

- <a href='https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/'>Computation graphs and Automatic Differentiation</a>
- <a href='https://blog.paperspace.com/pytorch-hooks-gradient-clipping-debugging/'>Hooks</a>
- <a href='https://cs231n.github.io/neural-networks-3/#loss'>Debugging Loss</a>
- <a href='http://karpathy.github.io/2019/04/25/recipe/'>A Recipe for training NN (Andrej Karpathy)</a>

https://wandb.ai/ayush-thakur/debug-neural-nets/reports/Visualizing-and-Debugging-Neural-Networks-with-PyTorch-and-W-B--Vmlldzo2OTUzNA

#### Objetivos:

Analizar de los mejores modelos los siguientes aspectos:

- Plot training y validation accuracy
- Analizar el error segun el learning rate
- Loss plot
- Activation function plot
- Plot activation/gradient histograms for all layers of the network
- Plot weights 1st layer

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_feature, ouput_classes, hidden_neurons=5):
        super().__init__()
        
        self.h1 = nn.Linear(in_features=input_feature,out_features=hidden_neurons)
        self.bn = nn.BatchNorm1d(hidden_neurons)
        self.out = nn.Linear(hidden_neurons,ouput_classes)

    def forward(self,x):
        # x = F.relu(self.h1(x))
        x = self.h1(x)
        x = F.relu(self.bn(x))
        # return self.out(x)    
        # return self.out(x)    
        return F.softmax(self.out(x),-1)   

    def reset_weights(self):
        self.h1.reset_parameters()
        self.bn.reset_parameters()
        self.out.reset_parameters()            

#### Training the model

In [ ]:
def prepareData():
    # Dimension reduction
    train_data      = ml.FootballMatchesDataset(file = 'train')
    train_data.data = train_data.data[:,:-2] # no consideramos ataque_defensa de toda la temporada
    features = dataDF.columns[:-2]

    # X_mean = torch.mean(train_data.data,dim=0).numpy()
    # X_norm = train_data.data / X_mean

    # # ANOVA: select best 7 features
    # anova7 = feature_selection.SelectKBest(score_func=feature_selection.f_classif,k=7)
    # anova7.fit(X_norm, train_data.labels.argmax(dim=1))
    # train_data.data = train_data.data[:,anova7.get_support()]
    # features = dataDF.columns[:-2][anova7.get_support()]
    train_data.data.shape
    return train_data, features

In [ ]:
importlib.reload(ml)


#### Train with Cross-Validation

In [ ]:
train_data, features = prepareData()

model = NeuralNetwork(train_data.data.shape[1],3)

lr=0.0001
bs=64
ep=100

old_data = train_data.data.clone()
scaler = preprocessing.Normalizer()
train_data.data = scaler.fit_transform(old_data).astype(np.float32)

title = f'mlp1x5_relu_sgd_anova07_norm_lr{lr}_epochs{ep}_b{bs}'
path_exec = path_logs+title+'//'

config = {
                    'net': NeuralNetwork, 'input': train_data.data.shape[1], 'output': 3, 
                    'hidden_neurons': 5, 'opt_name':'SGD', 'opt': torch.optim.SGD, 'lr': lr, 
                    'momentum': False, 'nesterov': False, 'criterion': nn.BCELoss, 
                    'bat_size': bs, 'epochs': ep
        }

er,ac_tr,ac_te,cm = ml.train_wCrossValidation(config, train_data, 
                        model_selection.KFold(n_splits=5,shuffle=True,random_state=0),
                        path=path_exec)

In [ ]:
fld = 3

with open(path_exec+f'f{fld}.json') as json_file:
    data = json.load(json_file)

    trainlogs   = pd.DataFrame(data['train'])
    testlogs    = pd.DataFrame(data['test'])

    importlib.reload(ml)
    ml.plot_error(trainlogs,path_exec,fld)

In [ ]:
trainlogs[trainlogs.it==0]

In [ ]:
importlib.reload(ml)

ml.plot_model_stats(path_exec,features,fld=3)

#### Only train model

In [ ]:
train_data      = ml.FootballMatchesDataset(file = 'train')
train_data.data = train_data.data[:,:-2] # no consideramos ataque_defensa de toda la temporada
features = dataDF.columns[:-2]

# normalization data
old_data = train_data.data.clone()
scaler = preprocessing.Normalizer()
train_data.data = scaler.fit_transform(old_data).astype(np.float32)

# split data

datatrain, datatest = (torch.utils.data.random_split(train_data, 
                            [1100-300,300], generator=torch.Generator().manual_seed(0)))

trainloader = DataLoader(datatrain,batch_size=64)
testloader  = DataLoader(datatest,batch_size=64)

In [ ]:
next(iter(trainloader))[0][:3]

In [ ]:
np.linalg.norm(train_data.data,axis=1)

In [ ]:
importlib.reload(ml)

model = NeuralNetwork(train_data.data.shape[1],3)

title = 'prueba'

er,ac_tr,ac_te,cm = ml.train_model(model,nn.BCELoss(),
                    torch.optim.SGD(lr=0.05,params=model.parameters()),
                    trainloader, testloader,    
                    epochs=150)

ml.save_log_model(title=title)

In [ ]:
with open(path_logs+title+'.json') as json_file:
    data = json.load(json_file)


In [ ]:
trainlogs   = pd.DataFrame(data['train'])
testlogs    = pd.DataFrame(data['test'])

In [ ]:
trainlogs.head(5)

In [ ]:
ml.plot_error(trainlogs,path_logs+title,0)

### ERROR AND ACCURACY PLOT

In [ ]:
plt.figure(figsize=(12,6))

for it in range(5):
    data = trainlogs[trainlogs.it==it]
    plt.plot(data.loss,label=it+1)

plt.title(f'Error model')
plt.xticks(range(0,len(trainlogs),250),rotation=45)
plt.legend(title='Batch')
# plt.grid()
plt.xlabel('iterations')
plt.ylabel('error')
plt.ylim([0.5,0.75])
# plt.savefig(path_exec + f'error_f{fld}.jpg', format='jpg', dpi=200)

plt.show()

In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(13,4))
testdata = testlogs[testlogs.it==0]
traindata = trainlogs[trainlogs.it==0]

fig.suptitle('Learning plot',fontsize=18)

ax1.plot(testlogs[testlogs.it==2].acc,color='#149AF8')

ax2.plot(trainlogs[trainlogs.it==3].acc,color='#FF774E')

ax1.set_title('Test accuracy'); ax2.set_title('Train accuracy')
# ax1.legend(title='Batch'); ax2.legend(title='Batch')
ax1.set_ylim([np.min(testdata.acc)-0.2,np.max(testdata.acc)+np.min(testdata.acc)])
ax2.set_ylim([np.min(testdata.acc)-0.2,np.max(testdata.acc)+np.min(testdata.acc)])

# plt.savefig(path_exec + f'accuracy_f{fld}.jpg', format='jpg', dpi=200)



In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=2,ncols=1,figsize=(10,12))
testdata = testlogs[testlogs.it==0]
traindata = trainlogs[trainlogs.it==0]

fig.suptitle('Learning plot',fontsize=20)

for it in range(max(testlogs.it)):
    data = testlogs[testlogs.it==it+1]
    ax1.plot(data.acc,label=it)

for it in range(5):
    data = trainlogs[trainlogs.it==it+1]
    ax2.plot(data.acc,label=it)

ax1.set_title('Test accuracy'); ax2.set_title('Train accuracy')
ax1.legend(title='Batch'); ax2.legend(title='Batch')
ax1.set_ylim([np.min(testdata.acc)-0.3,np.max(testdata.acc)+np.min(testdata.acc)])
ax2.set_ylim([np.min(testdata.acc)-0.3,np.max(testdata.acc)+np.min(testdata.acc)])

# plt.savefig(path_exec + f'accuracy_batches_f{fld}.jpg', format='jpg', dpi=200)



### WEIGHTS

In [ ]:
trainlogs.weights[0].keys()

In [ ]:
h1      = np.array([w.weights['h1.weight'] for w in trainlogs.itertuples()]).T
h1bias  = np.array([w.weights['h1.bias'] for w in trainlogs.itertuples()]).T
# h1 = h1.reshape(h1.shape[0],-1)
h1bias.shape

In [ ]:
fig = plt.figure(0,figsize=(25,12))
fig.suptitle('Hidden layer weights', fontsize=30)

for i in range(h1.shape[1]):
    ax = plt.subplot(2,3,i+1)
    ax.set_title(f'Weights unit {i}')
    for w,f in enumerate(features):
        if i==4: ax.plot(h1[w,i,:],label=f)
        else: ax.plot(h1[w,i,:])
    if i==4: ax.legend(title='Weights:',loc='lower center',bbox_to_anchor=(0.5, 1.05),
        ncol=3, fancybox=True, shadow=True)
    ax.grid()
    if i==0: ax.set_xlabel('iterations'); ax.set_ylabel('value')

ax = plt.subplot(230+h1.shape[1]+1)
ax.set_title('Biases of all units')
for i,bias in enumerate(h1bias):
    ax.plot(bias,label=i)
ax.legend(title='Unit:')
ax.grid()

# plt.savefig(path_exec + f'weights_f{fld}.jpg', format='jpg', dpi=200)

### GRADIENTS

In [ ]:
gradients = np.array(trainlogs.grad.to_list()).T
gradients.shape

In [ ]:
w = 0
n = 4

plt.figure(figsize=(9,6))
plt.plot(gradients[w,n,:],label=f'peso {w} unit {n}')
# plt.savefig(path_exec + f'gradient_f{fld}_w{w}_n{n}.jpg', format='jpg', dpi=200)

In [ ]:
fig = plt.figure(0,figsize=(25,12))
fig.suptitle('Hidden layer gradients', fontsize=30)

for i in range(gradients.shape[1]):
    ax = plt.subplot(2,3,i+1)
    ax.set_title(f'Gradients unit {i}')
    for w,f in enumerate(features):
        if i==4: ax.plot(gradients[w,i,:],label=f,alpha=0.3)
        else: ax.plot(gradients[w,i,:],alpha=0.5)
    ax.grid()
    if i==0: ax.set_xlabel('iterations'); ax.set_ylabel('value')
    if i==4: ax.legend(title='Gradients:',loc='lower center',bbox_to_anchor=(0.5, 1.05),
        ncol=3, fancybox=True, shadow=True)

# plt.savefig(path_exec + f'gradients_f{fld}.jpg', format='jpg', dpi=200)